In [29]:

from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_parse import LlamaParse
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import MarkdownElementNodeParser

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()

# API access to llama-cloud
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")
# Using Anthropic API for embeddings/LLMs
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [33]:


llm = Anthropic(model="claude-3-opus-20240229", temperature=0.0)

In [34]:
Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

In [35]:
# Parse 
parser = LlamaParse(
    api_key= LLAMA_CLOUD_API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction="""It contains specifications for different components. 
                            Reconstruct the information in a concise way.""",
    verbose=True
)

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader("./data_pistons", file_extractor=file_extractor).load_data()


Started parsing the file under job_id 71fa1fdb-eeec-4897-9974-bc73b4d04eab
.........Started parsing the file under job_id e2944c8f-c598-4219-9353-b5cf1ed2410c


In [36]:
# Nodes

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

166it [00:00, 55262.68it/s]
  0%|          | 0/166 [00:00<?, ?it/s]/Users/evanhymanson/Desktop/LlamaIndex/venv/lib/python3.12/site-packages/llama_index/core/program/utils.py:58: UserWarning: Failed to use `OpenAIPydanticProgram`. Please ensure that is installed by running `pip install llama-index-program-openai`.
  warnings.warn(
/Users/evanhymanson/Desktop/LlamaIndex/venv/lib/python3.12/site-packages/llama_index/core/program/utils.py:58: UserWarning: Failed to use `OpenAIPydanticProgram`. Please ensure that is installed by running `pip install llama-index-program-openai`.
  warnings.warn(
100%|██████████| 166/166 [07:46<00:00,  2.81s/it]
2it [00:00, 32263.88it/s]
  0%|          | 0/2 [00:00<?, ?it/s]/Users/evanhymanson/Desktop/LlamaIndex/venv/lib/python3.12/site-packages/llama_index/core/program/utils.py:58: UserWarning: Failed to use `OpenAIPydanticProgram`. Please ensure that is installed by running `pip install llama-index-program-openai`.
  warnings.warn(
100%|██████████| 2/2 [00:

In [37]:
# Reranker: most relevant docs
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
reranker = FlagEmbeddingReranker(top_n=5, model="BAAI/bge-reranker-large")

In [38]:
# Recursive Query Engine: for reading tables too 

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)
index = recursive_index
recursive_query_engine = recursive_index.as_query_engine(similarity_top_k=4, 
    node_postprocessors=[reranker], 
    verbose=True
)

In [39]:
# For printing response 
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core import StorageContext

In [ ]:
# Query 
#query = "Our client Audi wants to build a piston for their engine. Can you give me some recommendations? Here are the provided specifications: 1. ⁠Bore Diameter- 83.50 mm 2.⁠ ⁠Compression Height- 29.59 mm (1.165 inches) 3.⁠ ⁠Material- Forged 2618 T6 Aluminum 4.⁠ ⁠Compression Ratio- 8.5:1 5.⁠ ⁠Maximum G-Force- 8000 G's."
#response = recursive_query_engine.query(query)

In [40]:
from llama_index.core.schema import MetadataMode
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core import StorageContext

#pprint_response(response, show_source=True)

#print(response)

from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a manufacturing assistant. All numerical data that you encounter are specifications for components. You need to give recommendations of components that have the least numerical difference in specifications to the specifications provided by the user. The specifications of recommendations should also semantically be the most similar to the specifications provided by the user. "
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=False,
)

In [43]:

response = chat_engine.chat("Our client Audi wants to build a piston for their engine. Can you give me some recommendations? Here are the provided specifications: 1.⁠ ⁠Bore Diameter- 83.50 mm 2.⁠ ⁠Compression Height- 29.59 mm (1.165 inches) 3.⁠ ⁠Material- Forged 2618 T6 Aluminum 4.⁠ ⁠Compression Ratio- 8.5:1 5.⁠ ⁠Maximum G-Force- 8000 G's")

In [44]:
print(response)

Based on the specifications provided by Audi, I would recommend the following piston kits that have the closest matching specifications:

1. K450X3903 
- Bore: 3.903" (99.14 mm) - slightly larger than Audi's 83.50 mm bore
- Compression Ratio: 9.8 - 10.3 depending on combustion chamber volume. Closest to Audi's 8.5:1 spec.
- Pin Part #: 3905GFX (rated for 8000 G's max)

2. K0103A3
- Bore: 4.03" (102.36 mm) 
- Stroke: 3.5"
- Compression Ratio: 8.9 - 9.2. Very close to Audi's 8.5:1 spec.
- Pin Part #: W100F8-4040-5 
- Rings: Strutted

3. K0104A4
- Bore: 4.04" (102.62 mm)
- Stroke: 4.0" 
- Compression Ratio: 8.9 - 9.2
- Pin Part #: W100F8-4040-5
- Rings: Strutted

A few notes:
- The catalog doesn't specify materials, but these high performance kits are likely forged aluminum. You'd need to confirm 2618 T6 specifically.
- Compression height of 1.165" (29.59mm) is not listed for any kit. The closest is 1.050" for a couple 4.005" bore kits. Custom pistons may be needed to match this height ex

In [ ]:
# Node, metadata for final response 
print(response.source_nodes[0].metadata)